In [1]:
import os
import json
import random
from PIL import Image,ImageDraw
from scipy.spatial import distance

In [2]:
vg_image_dir = "/home/duypd/ThisPC-DuyPC/SG-Retrieval/Datasets/VisualGenome/VG_100K"

with open('/home/duypd/ThisPC-DuyPC/SG-Retrieval/Datasets/VisualGenome/anno_org/region_graphs.json') as f:
    rg2_json = json.load(f)

In [ ]:
with open('/home/duypd/ThisPC-DuyPC/SG-Retrieval/Datasets/VisualGenome/anno_org/scene_graphs.json') as f:
    sg_json = json.load(f)

with open('/home/duypd/ThisPC-DuyPC/SG-Retrieval/Datasets/VisualGenome/anno_org/region_descriptions.json') as f:
    rg_json = json.load(f)

with open('/home/duypd/ThisPC-DuyPC/SG-Retrieval/Datasets/VisualGenome/anno_org/image_data.json') as f:
    image_json = json.load(f)


# with open('/home/duypd/ThisPC-DuyPC/SG-Retrieval/CrossEncoderController/vg.json') as f:
#     vg = json.load(f)

In [5]:
#Information
print(len(sg_json))
print(sg_json[0].keys())

108077
dict_keys(['relationships', 'image_id', 'objects'])


In [7]:
#Information
print(len(rg_json))
print(rg_json[0].keys())
rg_json[0]

108077
dict_keys(['regions', 'id'])


{'regions': [{'region_id': 1382,
   'width': 82,
   'height': 139,
   'image_id': 1,
   'phrase': 'the clock is green in colour',
   'y': 57,
   'x': 421},
  {'region_id': 1383,
   'width': 182,
   'height': 109,
   'image_id': 1,
   'phrase': 'shade is along the street ',
   'y': 372,
   'x': 194},
  {'region_id': 1384,
   'width': 61,
   'height': 30,
   'image_id': 1,
   'phrase': 'man is wearing sneakers',
   'y': 491,
   'x': 241},
  {'region_id': 1385,
   'width': 36,
   'height': 36,
   'image_id': 1,
   'phrase': 'cars headlights are off',
   'y': 377,
   'x': 617},
  {'region_id': 1386,
   'width': 41,
   'height': 49,
   'image_id': 1,
   'phrase': 'bikes are parked at the far edge',
   'y': 298,
   'x': 322},
  {'region_id': 1432,
   'width': 84,
   'height': 178,
   'image_id': 1,
   'phrase': 'A sign on the facade of the building',
   'y': 14,
   'x': 122},
  {'region_id': 1433,
   'width': 96,
   'height': 332,
   'image_id': 1,
   'phrase': 'A tree trunk on the sidewalk'

In [2]:
print(len(rg2_json))
print(rg2_json[0].keys())
len(rg2_json[0]['regions'])
count  = 0
for item in rg2_json[0]['regions']:
    if(len(item['relationships'] == 0)):
       count = count + 1

count

NameError: name 'rg2_json' is not defined

In [54]:
combined = list(zip(sg_json, rg_json))  
random.shuffle(combined)  # Shuffle cả danh sách
sg_json, rg_json = zip(*combined)  # Tách lại thành hai danh sách

# Nếu cần danh sách thay vì tuple
sg_json = list(sg_json)
rg_json = list(rg_json)

c = []
for sg, rg in zip(sg_json, rg_json):
    if(sg['image_id'] == rg['id']):
        c.append(sg['image_id'])
    # break

In [55]:
len(c)

108077

In [56]:
with open('/home/duypd/ThisPC-DuyPC/SG-Retrieval/Datasets/VisualGenome/anno_org/check_example/updated_scene_graph.json') as f:
    tmp_anno = json.load(f)

image = Image.open(vg_image_dir + f"/{tmp_anno['image_id']}.jpg")
draw = ImageDraw.Draw(image)

for item in tmp_anno['objects'][:10]:
    x, y, w, h = item["x"], item["y"], item["w"], item["h"]
    phrase = item['names'][0]
    draw.rectangle([x, y, x + w, y + h], outline="blue", width=2)

    text_size = draw.textbbox((x, y), phrase)
    text_w, text_h = text_size[2] - text_size[0], text_size[3] - text_size[1]
    draw.rectangle([x, y - text_h - 5, x + text_w + 10, y], fill="black")
    draw.text((x + 5, y - text_h - 5), phrase, fill="white")

image.show()

In [57]:
combined = list(zip(sg_json, rg_json))  # Kết hợp hai danh sách thành một danh sách tuple
random.shuffle(combined)  # Shuffle cả danh sách
sg_json, rg_json = zip(*combined)  # Tách lại thành hai danh sách

# Nếu cần danh sách thay vì tuple
sg_json_new = list(sg_json)
rg_json_new = list(rg_json)

X = []
Y = []
X_b, Y_b = [], []

for x,y in zip(sg_json, rg_json):
    X.append(x['image_id'])
    Y.append(y['id'])

X == Y

True

In [58]:
objs = sg_json[0]['objects']
type(objs)
object_dict = {obj["object_id"]: obj for obj in objs}

In [59]:
def re_construct_sg(sg):
    objs = sg['objects']
    return_objs = []
    object_dict = {obj["object_id"]: obj for obj in objs}
    for item in sg['relationships']:
        return_obj = {}
        found_obj_sub = object_dict.get(item['subject_id'])
        found_obj_obj = object_dict.get(item['object_id'])
        return_obj["bbox_sub"] = [found_obj_sub['x'], found_obj_sub['y'], found_obj_sub['w'], found_obj_sub['h']]
        return_obj["bbox_obj"] = [found_obj_obj['x'], found_obj_obj['y'], found_obj_obj['w'], found_obj_obj['h']]
        return_obj["predicate"] = item["predicate"]
        return_obj["relationship_id"] = item["relationship_id"]
        return_objs.append(return_obj)
    return return_objs

def re_construct_rg(rg):
    return_objs = []
    for item in rg['regions']:
        return_obj = {}
        return_obj["bbox"] = [item['x'], item['y'], item['width'], item['height']]
        return_obj["phrase"] = item["phrase"]
        return_obj["region_id"] = item["region_id"]
        return_objs.append(return_obj)
    return return_objs

def get_image_data(image_id, image_data):
    for x in image_data:
        if(x['image_id'] == image_id):
            item = {
                'width': x['width'],
                'height': x['height'],
                'id': x['image_id'],
                'file_name': str(x['image_id'])+'.jpg'
            }
            return item

In [60]:
sg_json[0]['image_id']

2318366

In [61]:
return_objs_sg = re_construct_sg(sg_json[0])
return_objs_sg

[{'bbox_sub': [323, 245, 21, 17],
  'bbox_obj': [296, 237, 60, 85],
  'predicate': 'ON',
  'relationship_id': 4502150},
 {'bbox_sub': [58, 222, 19, 14],
  'bbox_obj': [3, 2, 495, 328],
  'predicate': 'flying in',
  'relationship_id': 4502151},
 {'bbox_sub': [356, 257, 142, 63],
  'bbox_obj': [392, 11, 62, 320],
  'predicate': 'on side of',
  'relationship_id': 4502152},
 {'bbox_sub': [249, 273, 15, 56],
  'bbox_obj': [19, 169, 474, 146],
  'predicate': 'IN',
  'relationship_id': 3976123},
 {'bbox_sub': [14, 256, 66, 75],
  'bbox_obj': [28, 275, 59, 55],
  'predicate': 'of a',
  'relationship_id': 3976125},
 {'bbox_sub': [186, 209, 193, 120],
  'bbox_obj': [261, 202, 104, 122],
  'predicate': 'OF',
  'relationship_id': 3976126},
 {'bbox_sub': [144, 23, 250, 307],
  'bbox_obj': [299, 0, 200, 331],
  'predicate': 'across',
  'relationship_id': 3976138}]

In [62]:
return_objs_rg = re_construct_rg(rg_json[0])
return_objs_rg

[{'bbox': [287, 209, 81, 121],
  'phrase': 'A red light on a traffic light ',
  'region_id': 4456067},
 {'bbox': [313, 240, 35, 29],
  'phrase': 'red traffic light ',
  'region_id': 4456077},
 {'bbox': [284, 229, 76, 102],
  'phrase': 'black traffic light ',
  'region_id': 4456078},
 {'bbox': [19, 41, 344, 290],
  'phrase': 'metal traffic sign support pole ',
  'region_id': 4456079},
 {'bbox': [382, 0, 67, 331],
  'phrase': 'tall metal rail tower ',
  'region_id': 4456080},
 {'bbox': [19, 269, 80, 61],
  'phrase': 'tall bare tree branches ',
  'region_id': 4456081},
 {'bbox': [78, 93, 418, 237],
  'phrase': 'electric power lines ',
  'region_id': 4456082},
 {'bbox': [357, 271, 32, 60],
  'phrase': 'upside down triangle street sign ',
  'region_id': 4456083},
 {'bbox': [74, 221, 76, 110],
  'phrase': 'tall street lamp ',
  'region_id': 4456084},
 {'bbox': [55, 221, 20, 15],
  'phrase': 'airplane flying in sky ',
  'region_id': 4456085},
 {'bbox': [352, 247, 145, 82],
  'phrase': 'stone 

In [63]:
def compute_iou(bbox1, bbox2):
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2

    # Tọa độ của phần giao nhau
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)

    # Nếu không có giao nhau, IoU = 0
    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # Tính diện tích giao nhau
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Tính diện tích của từng bbox
    bbox1_area = w1 * h1
    bbox2_area = w2 * h2

    # Tính IoU
    iou = intersection_area / float(bbox1_area + bbox2_area - intersection_area)
    return iou
region_mappings = []
for region in return_objs_rg:
    x, y, w, h = region["bbox"]
    center_x = x + w / 2
    center_y = y + h / 2
    region_mappings.append({"center": (center_x, center_y), "phrase": region["phrase"], "bbox": region["bbox"]})

region_mappings

[{'center': (327.5, 269.5),
  'phrase': 'A red light on a traffic light ',
  'bbox': [287, 209, 81, 121]},
 {'center': (330.5, 254.5),
  'phrase': 'red traffic light ',
  'bbox': [313, 240, 35, 29]},
 {'center': (322.0, 280.0),
  'phrase': 'black traffic light ',
  'bbox': [284, 229, 76, 102]},
 {'center': (191.0, 186.0),
  'phrase': 'metal traffic sign support pole ',
  'bbox': [19, 41, 344, 290]},
 {'center': (415.5, 165.5),
  'phrase': 'tall metal rail tower ',
  'bbox': [382, 0, 67, 331]},
 {'center': (59.0, 299.5),
  'phrase': 'tall bare tree branches ',
  'bbox': [19, 269, 80, 61]},
 {'center': (287.0, 211.5),
  'phrase': 'electric power lines ',
  'bbox': [78, 93, 418, 237]},
 {'center': (373.0, 301.0),
  'phrase': 'upside down triangle street sign ',
  'bbox': [357, 271, 32, 60]},
 {'center': (112.0, 276.0),
  'phrase': 'tall street lamp ',
  'bbox': [74, 221, 76, 110]},
 {'center': (65.0, 228.5),
  'phrase': 'airplane flying in sky ',
  'bbox': [55, 221, 20, 15]},
 {'center': 

In [75]:
def nearest_phase_bbox(bbox, region_mappings, iou_threshold=0.5):

    best_match = None
    best_iou = 0

    for region in region_mappings:
        region_bbox = region["bbox"]

        # Tính IoU giữa scene graph bbox và region bbox
        iou = compute_iou(bbox, region_bbox)

        # Nếu IoU cao nhất, cập nhật best_match
        if iou > best_iou:
            best_iou = iou
            best_match = region

    # Nếu không có IoU đủ cao, trả về "Unknown"
    return best_match if best_match and best_iou > iou_threshold else {"phrase": "Unknown"}

for i in return_objs_sg:
    phase_sub = nearest_phase_bbox(i['bbox_sub'], region_mappings)
    phase_obj = nearest_phase_bbox(i['bbox_obj'], region_mappings)
    if(phase_sub == phase_obj or phase_sub['phrase'] == 'Unknown' or phase_obj['phrase'] == 'Unknown'):
        continue
    print(i)
    print(phase_sub)
    print(phase_obj)
    print(f"{phase_sub['phrase']} - {i['predicate']} - {phase_obj['phrase']}")
    print(3*"===============================")

{'bbox_sub': [58, 222, 19, 14], 'bbox_obj': [3, 2, 495, 328], 'predicate': 'flying in', 'relationship_id': 4502151}
{'center': (68.0, 228.5), 'phrase': 'Airplane that is flying in the sky', 'bbox': [60, 222, 16, 13]}
{'center': (191.0, 186.0), 'phrase': 'metal traffic sign support pole ', 'bbox': [19, 41, 344, 290]}
Airplane that is flying in the sky - flying in - metal traffic sign support pole 
{'bbox_sub': [356, 257, 142, 63], 'bbox_obj': [392, 11, 62, 320], 'predicate': 'on side of', 'relationship_id': 4502152}
{'center': (425.5, 293.0), 'phrase': 'tall building on right', 'bbox': [353, 256, 145, 74]}
{'center': (415.5, 165.5), 'phrase': 'tall metal rail tower ', 'bbox': [382, 0, 67, 331]}
tall building on right - on side of - tall metal rail tower 
{'bbox_sub': [14, 256, 66, 75], 'bbox_obj': [28, 275, 59, 55], 'predicate': 'of a', 'relationship_id': 3976125}
{'center': (46.5, 299.5), 'phrase': 'a part of the tree', 'bbox': [11, 271, 71, 57]}
{'center': (59.0, 299.5), 'phrase': 'ta

In [20]:
im = Image.open(vg_image_dir + f"/{rg2_json[0]['image_id']}.jpg")
draw = ImageDraw.Draw(im)
draw.rectangle([194, 372, 194 + 182, 372 + 109], outline="blue", width=2)
draw.rectangle([119, 338, 119 + 274, 338 + 192], outline="red", width=2)
draw.rectangle([77, 328, 77 + 714, 328 + 262], outline="orange", width=2)
im.show()

In [3]:
with open('/home/duypd/ThisPC-DuyPC/SG-Retrieval/CrossEncoderController/vg_val.json') as f:
        check_val = json.load(f)

In [5]:
check_val.keys()

dict_keys(['images', 'annotations'])

In [4]:
check_val['images'][0].keys()

len(check_val['images'])

10480

In [5]:
len(check_val['annotations'])

231364

In [21]:
check_val['annotations'][0]

{'image_id': 176,
 'iscrowd': 0,
 'id': 1855276,
 'category_id': 1,
 'segmentation': None,
 'area_sub': 14091,
 'area_obj': 108640,
 'bbox_sub': [603, 519, 183, 77],
 'bbox_obj': [95, 437, 679, 160],
 'phrase_sub': 'Walkway made of pavers',
 'phrase_obj': 'woman taking a picture of a younger woman',
 'predicate': 'ON',
 'predicate_id': 3188889}

In [7]:
img = []
for item in check_val['annotations']:
    if(item['image_id'] not in img):
        img.append(item['image_id'])

In [8]:
len(img)

10480

In [15]:
import gc
from collections import defaultdict

annotation_dict = defaultdict(list)
for item in check_val['annotations']:
    annotation_dict[item['image_id']].append(item)

def get_target(annotations):
    """Hàm xử lý annotation cho một ảnh."""
    return {
        "image_id": annotations[0]['image_id'],
        "category_id": [item['category_id'] for item in annotations],
        "segmentation": [item['segmentation'] for item in annotations],
        "area_sub": [item['area_sub'] for item in annotations],
        "area_obj": [item['area_obj'] for item in annotations],
        "bbox_sub": [item['bbox_sub'] for item in annotations],
        "bbox_obj": [item['bbox_obj'] for item in annotations],
        "phrase_sub": [item['phrase_sub'] for item in annotations],
        "phrase_obj": [item['phrase_obj'] for item in annotations],
        "predicate": [item['predicate'] for item in annotations],
        "predicate_id": [item['predicate_id'] for item in annotations],
    }

# Danh sách chỉ lưu index, không lưu toàn bộ ảnh vào bộ nhớ
image_idx = []
tgt_idx = []

for img_info in check_val['images']:
    im = Image.open(vg_image_dir + '/' + img_info['file_name']).convert("RGB")
    image_idx.append(im)
    
    # Chỉ lưu đường dẫn thay vì toàn bộ ảnh
    # image_idx.append(image_path)

    # Lấy annotations tương ứng từ annotation_dict (không cần duyệt toàn bộ danh sách)
    annotations = annotation_dict.get(img_info['id'], [])
    
    # Nếu ảnh có annotations, lưu target
    if annotations:
        tgt_idx.append(get_target(annotations))
    
    # Giải phóng bộ nhớ sau mỗi vòng lặp
    gc.collect()

    # break




Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x757ad9914040>>
Traceback (most recent call last):
  File "/home/duypd/ThisPC-DuyPC/SG-Retrieval/.venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [17]:
import gc
from collections import defaultdict

annotation_dict = defaultdict(list)
for item in check_val['annotations']:
    annotation_dict[item['image_id']].append(item)

def get_annotations_by_image_id(image_id):
    return annotation_dict.get(image_id, [])

In [22]:
annotation_dict.keys()

dict_keys([176, 2318039, 2382617, 2380648, 286048, 2416299, 2341850, 2359896, 2560, 2377023, 2393393, 2362935, 2320812, 2323268, 2361018, 2404608, 2319361, 2359156, 2385651, 498265, 2366289, 2386446, 2407067, 2367542, 2358856, 2380211, 2356139, 2411667, 2408858, 2409440, 2356812, 2325002, 2368435, 2339339, 2348894, 2382181, 2389806, 2406847, 2381455, 2405445, 1215, 2396961, 2330922, 2318305, 1592885, 2407380, 2333735, 2395401, 2622, 2356516, 2327417, 2381428, 2328131, 2324064, 2352623, 2333920, 2403121, 2358138, 2390743, 2399722, 2391121, 2383669, 2324767, 2391208, 2340846, 2355321, 1997, 2358368, 2318997, 2408078, 2335165, 2340723, 2332804, 2356157, 2378793, 2316392, 2391913, 2411466, 2324522, 2366659, 2320200, 2350865, 2360487, 2325607, 2356378, 2323373, 2410930, 1159320, 2377439, 2344526, 2318076, 2331579, 2343887, 2350044, 2391507, 713503, 2399347, 61560, 2394709, 2333595, 2336817, 2380475, 2360467, 2361725, 2368124, 2389134, 2412384, 2341742, 2358336, 2328709, 2382956, 2404910, 23

In [30]:
matched_annotations = get_annotations_by_image_id(176)
matched_annotations


[{'image_id': 176,
  'iscrowd': 0,
  'id': 1855276,
  'category_id': 1,
  'segmentation': None,
  'area_sub': 14091,
  'area_obj': 108640,
  'bbox_sub': [603, 519, 183, 77],
  'bbox_obj': [95, 437, 679, 160],
  'phrase_sub': 'Walkway made of pavers',
  'phrase_obj': 'woman taking a picture of a younger woman',
  'predicate': 'ON',
  'predicate_id': 3188889},
 {'image_id': 176,
  'iscrowd': 0,
  'id': 1855277,
  'category_id': 1,
  'segmentation': None,
  'area_sub': 30258,
  'area_obj': 23870,
  'bbox_sub': [280, 320, 246, 123],
  'bbox_obj': [305, 308, 217, 110],
  'phrase_sub': 'large fountain in front of a building',
  'phrase_obj': 'large fountain in front of a building',
  'predicate': 'has',
  'predicate_id': 3188890},
 {'image_id': 176,
  'iscrowd': 0,
  'id': 1855278,
  'category_id': 1,
  'segmentation': None,
  'area_sub': 68697,
  'area_obj': 108640,
  'bbox_sub': [8, 60, 153, 449],
  'bbox_obj': [95, 437, 679, 160],
  'phrase_sub': 'Palm trees lining the walkway',
  'phrase

In [33]:
def get_target(annotations):
    """Hàm xử lý annotation cho một ảnh."""
    return {
        "image_id": annotations[0]['image_id'],
        "category_id": [item['category_id'] for item in annotations],
        "segmentation": [item['segmentation'] for item in annotations],
        "area_sub": [item['area_sub'] for item in annotations],
        "area_obj": [item['area_obj'] for item in annotations],
        "bbox_sub": [item['bbox_sub'] for item in annotations],
        "bbox_obj": [item['bbox_obj'] for item in annotations],
        "phrase_sub": [item['phrase_sub'] for item in annotations],
        "phrase_obj": [item['phrase_obj'] for item in annotations],
        "predicate": [item['predicate'] for item in annotations],
        "predicate_id": [item['predicate_id'] for item in annotations],
    }

x = get_target(matched_annotations)
x

{'image_id': 176,
 'category_id': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'segmentation': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'area_sub': [14091,
  30258,
  68697,
  1953,
  1240,
  7936,
  30258,
  30258,
  163586,
  163586,
  21222,
  21222,
  163586,
  163586,
  21222,
  21222,
  1488,
  1488,
  163586,
  1449,
  1488,
  21222,
  1488,
  1488,
  3375,
  1953,
  2340,
  5304],
 'area_obj': [108640,
  23870,
  108640,
  30258,
  30258,
  60621,
  23870,
  23870,
  21222,
  21222,
  163586,
  163586,
  8556,
  5085,
  163586,
  163586,
  163586,
  163586,
  1449,
  163586,
  163586,
  163586,
  163586,
  163586,
  23870,
  756,
  30258,
  910],
 'bbox_sub': [[603, 519, 183, 77],
  [280, 320, 246, 123]